# Model Training: Fine-tuning xLAM-2 for Banking Agent

This notebook demonstrates the fine-tuning process for the xLAM-2-1b-fc-r model on banking agent tasks.

In [ ]:
import sys
sys.path.append('../src')

from data_preparation import BankingDatasetProcessor
from training import BankingAgentTrainer, ModelArguments, DataArguments
import torch
import wandb

## Step 1: Data Preparation

In [ ]:
# Initialize data processor
processor = BankingDatasetProcessor()

# Prepare dataset
output_dir = processor.prepare_for_training("../data/processed")
print(f"Dataset prepared in: {output_dir}")

## Step 2: Model Configuration

In [ ]:
# Configure training arguments
model_args = ModelArguments(
    model_name_or_path="Salesforce/xLAM-2-1b-fc-r",
    use_lora=True,
    lora_r=16,
    lora_alpha=32,
    lora_dropout=0.1
)

data_args = DataArguments(
    data_path="../data/processed",
    max_length=2048
)

print("Training configuration:")
print(f"Model: {model_args.model_name_or_path}")
print(f"LoRA enabled: {model_args.use_lora}")
print(f"Max length: {data_args.max_length}")

## Step 3: Initialize Trainer

In [ ]:
# Initialize trainer
trainer = BankingAgentTrainer(model_args, data_args)

# Setup model and tokenizer
trainer.setup_model_and_tokenizer()

print("Trainer initialized successfully!")
print(f"Model device: {trainer.model.device}")
print(f"Model dtype: {trainer.model.dtype}")

## Step 4: Load and Inspect Training Data

In [ ]:
# Load datasets
datasets = trainer.load_dataset()

print("Dataset sizes:")
for split, dataset in datasets.items():
    print(f"{split}: {len(dataset)} examples")

# Show sample training example
print("\nSample training example:")
sample = datasets['train'][0]
print(f"Text length: {len(sample['text'])} characters")
print(f"Scenario ID: {sample['scenario_id']}")
print(f"Persona Index: {sample['persona_index']}")
print(f"\nFirst 500 characters:")
print(sample['text'][:500] + "...")

## Step 5: Start Training (Optional - can be resource intensive)

In [ ]:
# Note: This cell is optional and can be resource-intensive
# Uncomment to run actual training

# # Initialize wandb (optional)
# wandb.init(project="xlam-banking-agent", name="notebook-training")

# # Start training
# output_dir = "../models/xlam-banking-agent"
# trained_model = trainer.train(output_dir)

# print(f"Training completed! Model saved to {output_dir}")

print("Training cell is commented out to avoid resource usage.")
print("Uncomment the code above to run actual training.")
print("Alternatively, use the command line script: bash scripts/run_training.sh")

## Step 6: Training Monitoring Tips

In [ ]:
print("Training Monitoring Tips:")
print("1. Monitor GPU memory usage with: nvidia-smi")
print("2. Watch training logs in real-time: tail -f logs/training.log")
print("3. Use wandb dashboard for metrics visualization")
print("4. Expected training time: 2-4 hours on single GPU")
print("5. Watch for overfitting - validation loss should decrease")

print("\nRecommended hyperparameters for different setups:")
print("- Single GPU (8GB): batch_size=2, gradient_accumulation=8")
print("- Single GPU (16GB): batch_size=4, gradient_accumulation=4")
print("- Multi-GPU: Increase batch_size proportionally")